## Code Sources

- https://towardsdatascience.com/image-feature-extraction-using-pytorch-e3b327c3607a

- https://github.com/MnCSSJ4x/VR-MiniProject/blob/main/VR3b.ipynb

- https://colab.research.google.com/github/ashishpatel26/Awesome-Pytorch-Tutorials/blob/main/17.Pytorch%20Transfer%20learning%20with%20Caltech101.ipynb

## CUDA

In [1]:
from torch import cuda
from torch import device

In [2]:
'cuda:0' if cuda.is_available() else 'cpu'

'cuda:0'

In [3]:
device = device('cuda:0' if cuda.is_available() else 'cpu')

## AlexNet

In [4]:
from torchvision.models import alexnet

In [5]:
alexnet = alexnet(weights='AlexNet_Weights.DEFAULT')
print(alexnet)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

## Feature Extractor

In [6]:
from torch import nn
from torchsummary import summary

In [7]:
class FeatureExtractor(nn.Module):

  def __init__(self, model):
    super(FeatureExtractor, self).__init__()
    # Extract Feature Layers
    self.features = model.features
    # Extract Average Pooling Layer
    self.pooling = model.avgpool
    # Convert the image into one-dimensional vector
    self.flatten = nn.Flatten()
  
  def forward(self, x):
    # Take image x and return a feature vector
    out = self.features(x)
    out = self.pooling(out)
    out = self.flatten(out)
    return out 

feature_extractor = FeatureExtractor(alexnet)
feature_extractor = feature_extractor.to(device)

In [8]:
print(summary(feature_extractor, input_size=(3, 224, 224)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [9]:
import tarfile
from imutils import paths
import os
import cv2
import numpy as np
from torchvision import transforms
import pandas as pd
import torch

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

## Extract Features for Bikes vs Horses Dataset

In [19]:
from imutils import paths
import os
import cv2
import numpy as np
import pandas as pdy

In [20]:
!unzip '/content/drive/MyDrive/Assignment2_BikeHorses.zip'

Archive:  /content/drive/MyDrive/Assignment2_BikeHorses.zip
   creating: Assignment2_BikeHorses/
   creating: Assignment2_BikeHorses/Bikes/
  inflating: Assignment2_BikeHorses/Bikes/0042.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0023.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0004.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0066.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0047.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0028.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0071.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0052.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0009.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0033.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0014.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0076.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0057.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0038.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0019.jpg  
  inflating: Assignment2_BikeHorses/Bikes/0062.jpg  
  inflating:

In [21]:
images = []
labels = []

In [22]:
image_paths = list(paths.list_images('./Assignment2_BikeHorses/Horses'))

In [23]:
for img_path in image_paths:
    label = 'horse'
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    labels.append(label)

In [24]:
image_paths = list(paths.list_images('./Assignment2_BikeHorses/Bikes'))

In [25]:
for img_path in image_paths:
    label = 'bike'
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    labels.append(label)

In [26]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

In [27]:
features_array = []

for i in range(len(images)):
    img = images[i]
    img = transform(img)
    img = img.reshape(1, 3, 224, 224)
    img = img.to(device)
    with torch.no_grad():
        img_features = feature_extractor(img)
    img_features = img_features.cpu().detach().numpy().reshape(-1)
    features_array.append(img_features)

In [28]:
np.array(features_array).shape

(179, 9216)

In [29]:
X = np.array(features_array)
y = np.array(labels)
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 2.6343925 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        2.3295362 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.19934064]], dtype=float32)

In [30]:
df = pd.DataFrame(X)
df['label'] = y

In [31]:
X=df.drop("label",axis=1)
y=df["label"]
import pickle
# Open a file and use dump()
with open('X_bikevshorse.pkl', 'wb') as file:
    pickle.dump(X, file)
    
with open('y_bikevshorse.pkl', 'wb') as file:
    pickle.dump(y, file)

In [32]:
import pickle
  
# Open the file in binary mode
with open('X_bikevshorse.pkl', 'rb') as file:
    # Call load method to deserialze
    X = pickle.load(file)
with open('y_bikevshorse.pkl', 'rb') as file:
    # Call load method to deserialze
    y = pickle.load(file)

In [33]:
X

,0,1,2,3,4,5,6,7,8,9,...,9206,9207,9208,9209,9210,9211,9212,9213,9214,9215
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000
1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,2.534982,0.0,0.0,0.000000,1.051755,3.843845,5.14584,2.634392,0.0,0.000000
2,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000
3,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000
4,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.930111,0.000000,0.000000,0.00000,0.000000,0.0,1.294524
175,0.0,0.0,0.0,2.279662,7.933328,7.933328,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000
176,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000
177,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,2.329536


## Model

In [34]:
df

,0,1,2,3,4,5,6,7,8,9,...,9207,9208,9209,9210,9211,9212,9213,9214,9215,label
0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,horse
1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,1.051755,3.843845,5.14584,2.634392,0.0,0.000000,horse
2,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,horse
3,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,horse
4,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,horse
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.930111,0.000000,0.000000,0.00000,0.000000,0.0,1.294524,bike
175,0.0,0.0,0.0,2.279662,7.933328,7.933328,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,bike
176,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,bike
177,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,2.329536,bike


In [35]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [36]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=150)
X_pca = pca.fit_transform(X_scaled)

In [37]:
pca.explained_variance_ratio_.sum()

0.96871257

In [38]:
df_pca = pd.DataFrame(X_pca)
df_pca['label'] = labels

In [39]:
label_encoder = LabelEncoder()
df_pca['label']= label_encoder.fit_transform(df_pca['label'])

In [40]:
df_pca

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,label
0,-32.951469,-2.889146,-14.803780,10.899224,13.684970,-0.181006,-5.243152,8.348410,-15.832732,1.459659,...,0.247990,-0.353771,-0.672298,0.658493,1.179097,1.249306,-0.370977,-0.108364,-1.327536,1
1,-24.712564,-4.768534,-10.869353,-14.198569,-19.957775,-1.628917,12.494553,-2.068352,-4.399955,-10.265526,...,2.351405,2.575213,-1.487175,2.025034,1.393975,-0.819217,0.477292,-0.419664,1.401235,1
2,-34.304516,-4.618460,-6.428257,-12.861193,11.890509,0.527339,-7.304446,7.746315,-4.622585,-8.189036,...,3.433625,6.090095,1.261150,-6.140683,0.978930,2.089636,-1.125864,0.514659,-1.424337,1
3,-21.398035,2.832531,7.416690,5.745567,14.358952,-13.286668,-0.709282,2.649159,14.406150,-13.657684,...,-0.995189,0.526243,-0.542420,2.057589,-0.381966,-0.071647,0.216545,-1.428892,0.390479,1
4,-30.401480,-2.721661,-0.811812,5.819017,22.800846,5.050194,-0.115094,16.021645,-11.156420,0.924892,...,-1.253929,-1.632631,0.517950,-1.970316,0.171981,-1.126902,0.304872,-1.398370,0.677529,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,33.001194,14.904472,-20.126928,-0.090137,-12.052032,5.785666,13.507404,-2.847554,9.641938,-0.682474,...,0.819405,-4.929142,-1.061009,7.081267,-11.251348,-0.751163,5.571714,1.103489,0.324901,0
175,17.994638,23.365463,47.050686,-2.723074,-4.947339,-26.360069,-12.322076,15.521224,6.114455,3.020345,...,1.213910,-0.373187,-0.048279,-0.348113,0.138783,-0.539702,0.096508,0.037717,-0.754466,0
176,27.387442,-36.957840,7.843657,-0.543022,-4.062580,4.528579,-5.559773,12.044427,18.093517,5.464409,...,0.193216,-1.939827,5.733002,1.808512,7.920508,-0.864252,10.863525,-2.905275,4.316067,0
177,28.281246,9.186474,-20.520117,-1.075391,-1.386285,0.855687,9.533030,-3.850548,0.007347,1.264727,...,-2.588512,-8.974262,0.919534,3.599506,11.006666,2.324006,-12.135151,-9.596021,-3.537851,0


In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X_train.shape

(89, 9216)

In [42]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=100000).fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.score(X_test,y_test)

1.0

In [43]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel='linear'))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

1.0

In [44]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',kernel='rbf'))
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

1.0

In [45]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10, random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

1.0

In [46]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

1.0

In [47]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=50, random_state=42)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

1.0

In [48]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=20, random_state=42,n_estimators=500)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

1.0

In [49]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
neigh.score(X_test,y_test)

1.0